In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression



%matplotlib inline

### Parameters for plotting model results ###
pd.set_option("display.max_colwidth",100)
sns.set(style="ticks", color_codes=True)
plt.rcParams['font.weight'] = 'normal'
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['axes.labelpad'] = 5
plt.rcParams['axes.linewidth']= 2
plt.rcParams['xtick.labelsize']= 14
plt.rcParams['ytick.labelsize']= 14
plt.rcParams['xtick.major.size'] = 6
plt.rcParams['ytick.major.size'] = 6
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.minor.width'] = 1
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['axes.edgecolor'] = 'black'

def one_hot_encode(df, col='utr', seq_len=50):
    # Dictionary returning one-hot encoding of nucleotides. 
    nuc_d = {'a':[1,0,0,0],'c':[0,1,0,0],'g':[0,0,1,0],'t':[0,0,0,1], 'n':[0,0,0,0]}
    
    # Creat empty matrix.
    vectors=np.empty([len(df),seq_len,4])
    
    # Iterate through UTRs and one-hot encode
    for i,seq in enumerate(df[col].str[:seq_len]): 
        seq = seq.lower()
        a = np.array([nuc_d[x] for x in seq])
        vectors[i] = a
    return vectors


def r2(x,y):
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    return r_value**2

def valid_predict(train_df, valid_df, model, test_seq, obs_col, output_col='pred'):
    '''Predict mean ribosome load using model and test set UTRs'''
    
    # Scale the test set mean ribosome load
    scaler = preprocessing.StandardScaler()
    scaler.fit(train_df[obs_col].values.reshape(-1,1))
    
    # Make predictions
    predictions = model.predict(test_seq).reshape(-1,1)
    
    # Inverse scaled predicted mean ribosome load and return in a column labeled 'pred'
    df_copy = valid_df.copy()
    df_copy.loc[:,output_col] = scaler.inverse_transform(predictions)
    return df_copy

### Load data, make train and test sets with same proportion of uAUG UTRs
The test set used by the authors contains UTRs with the highest overall sequencing reads with the idea that increased reads will more accurately reflect the true ribosome load of a given 5'UTR. However, sequences with a uAUG are enriched in the set of 20,000 UTRs with the most reads, so we'll split in a way that keeps the proportion of uATG the same.

In [ ]:
train = pd.read_csv('../data/egfp_unmod_1_split/train.csv')
valid = pd.read_csv('../data/egfp_unmod_1_split/valid.csv')
train_one_hot = np.load('../data/egfp_unmod_1_split/onehot_train.npy')
valid_one_hot = np.load('../data/egfp_unmod_1_split/onehot_valid.npy')

In [ ]:
train_one_hot = train_one_hot.reshape(224000, 200)
valid_one_hot = valid_one_hot.reshape(28000, 200)


### Train model
Using the hyperparameter-optimised values.

In [ ]:
reg = LinearRegression().fit(train_one_hot, train['scaled_rl'])
reg.score(train_one_hot, train['scaled_rl'])

Evaluate model. Return predicted mean ribosome load as a dataframe column labeled 'pred'.

In [ ]:
valid = valid_predict(train_df=train, valid_df=valid, model=reg, obs_col='rl',test_seq=valid_one_hot)
pred = reg.predict(valid_one_hot)
r = r2(valid['scaled_rl'], valid['pred'])
print('r-squared = ', r)

### Plotting test results

In [ ]:
atg = valid[valid['utr'].apply(lambda x: 'ATG' in x)]
n_atg = valid[valid['utr'].apply(lambda x: 'ATG' not in x)]

Test r-squared for uATG and no uATG sequences separately

In [ ]:
print("uATG: ", r2(atg['rl'], atg['pred']))
print("no uATG: ", r2(n_atg['rl'], n_atg['pred']))

In [ ]:
c1 = (0.3, 0.45, 0.69)
c2 = 'r'
g = sns.JointGrid(x='rl', y="pred", data=atg, space=0, xlim=(1,10), ylim=(0,10), ratio=6, height=7)
g.plot_joint(plt.scatter,s=20, color=c1, linewidth=0.2, alpha=0.1, edgecolor='white')
f = g.fig
ax = f.gca()
ax.set_yticks(np.arange(0,9.01, 1))
ax.set_yticklabels(range(10),size=20)
ax.set_xticks(np.arange(1,10.01, 1))
ax.set_xticklabels(range(1,11),size=20)
ax.set_ylim(0,9)
ax.set_xlim(1,10)
g.plot_marginals(sns.kdeplot,shade=c1, **{'linewidth':2, 'color':c1})
g.set_axis_labels('Observed MRL', 'Predicted MRL', **{'size':22});

g.x = n_atg['rl'].values
g.y = n_atg['pred'].values
g.plot_joint(plt.scatter, s=20, linewidth=0.2, alpha=0.2, color=c2, edgecolor='white')
g.plot_marginals(sns.kdeplot, shade=c2, **{'linewidth':2, 'color':c2})
f = g.fig